# 🎯 Mini-Projet : Décodeur Morse par Clignement des Yeux avec dlib et YOLOv8

---

## 📝 Présentation du projet

Ce mini-projet explore deux approches pour détecter et décoder le clignement des yeux en code Morse, permettant une communication alternative basée sur les mouvements oculaires.  

- **Première partie** : Utilisation de la bibliothèque **dlib** pour la détection des yeux, combinée à un réseau de neurones convolutionnel (**CNN**) pré-entraîné qui classifie l'état des yeux (ouverts/fermés).  
- **Deuxième partie** : Remplacement de dlib par un modèle plus performant et rapide, **YOLOv8**, également couplé au même CNN pour la classification. Ce modèle YOLOv8 est lui aussi pré-entraîné et sauvegardé.

---

## ⚙️ Fonctionnement

- **Détection des yeux** :  
  - Dans la première approche, dlib localise les yeux sur le visage capturé.  
  - Dans la seconde approche, YOLOv8 détecte les yeux plus rapidement et avec plus de précision.  
- **Classification de l’état des yeux** : Pour chaque œil détecté, un CNN pré-entraîné identifie s’il est ouvert ou fermé.  
- **Analyse des clignements** : Lorsque les deux yeux sont fermés simultanément, un clignement est détecté.  
- **Décodage temporel Morse** : La durée du clignement définit le symbole Morse correspondant (point, tiret, espace entre lettres ou mots).  
- **Interface utilisateur** : Retour visuel en temps réel avec affichage du statut des yeux, du texte décodé, du buffer Morse, et commandes clavier pour contrôler l’application (quitter, réinitialiser, sauvegarder, effacer).

---

Ce projet illustre la synergie entre techniques classiques (dlib) et modernes (YOLOv8) de vision par ordinateur, associées à un CNN pour classification.


In [233]:
import cv2
import dlib
import numpy as np
import time
from collections import deque
from tensorflow.keras.models import load_model
import winsound  

class EyeBlinkMorseDecoder:
    """Decode eye‑blinks into Morse code, with basic editing (backspace) support."""

    def __init__(self):
        # ----------------------- Model & detectors ----------------------- #
        self.model = load_model("eye_blink_cnn_model.h5", compile=False)
        self.detector = dlib.get_frontal_face_detector()
        self.predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

        # Eye landmarks (dlib 68‑pt modelqb)
        self.LEFT_EYE = list(range(36, 42))
        self.RIGHT_EYE = list(range(42, 48))

        # ---------------------- Morse dictionaries ---------------------- #
        self.MORSE_CODE_DICT = {
            '.-': 'A', '-...': 'B', '-.-.': 'C', '-..': 'D',
            '.': 'E', '..-.': 'F', '--.': 'G', '....': 'H',
            '..': 'I', '.---': 'J', '-.-': 'K', '.-..': 'L',
            '--': 'M', '-.': 'N', '---': 'O', '.--.': 'P',
            '--.-': 'Q', '.-.': 'R', '...': 'S', '-': 'T',
            '..-': 'U', '...-': 'V', '.--': 'W', '-..-': 'X',
            '-.--': 'Y', '--..': 'Z', '-----': '0', '.----': '1',
            '..---': '2', '...--': '3', '....-': '4', '.....': '5',
            '-....': '6', '--...': '7', '---..': '8', '----.': '9'
        }

        # ---------------------- Timing thresholds ---------------------- #
        self.DOT_MAX = 0.6      # Short blink = dot
        self.DASH_MIN = 1.0     # Medium blink = dash
        self.LETTER_SEP = 1.8   # Long blink  = letter separator
        self.WORD_SEP = 2.5     # Extra‑long  = word separator

        # ------------------------ Audio tones -------------------------- #
        self.BEEP_FREQ_SHORT = 800  # Hz – dot / confirmation
        self.BEEP_FREQ_LONG = 600   # Hz – dash
        self.BEEP_DURATION = 100    # ms

        # ---------------------- Runtime state -------------------------- #
        self.blink_start_time = None
        self.current_state = "IDLE"  # or "BLINKING"
        self.morse_buffer: list[str] = []  # Raw symbols
        self.text_buffer = ""            # Decoded text
        self.message_history = deque(maxlen=10)

        # Visual feedback helpers
        self.last_blink_duration = 0
        self.last_blink_type = ""
        self.feedback_text = ""
        self.feedback_timer = 0
        self.face_detected = False

    # ------------------------------------------------------------------ #
    #                          Helper methods                            #
    # ------------------------------------------------------------------ #

    def play_sound(self, blink_type: str):
        """Play a small beep to acknowledge a blink‑type."""
        try:
            if blink_type == "DOT":
                winsound.Beep(self.BEEP_FREQ_SHORT, self.BEEP_DURATION)
            elif blink_type == "DASH":
                winsound.Beep(self.BEEP_FREQ_LONG, int(self.BEEP_DURATION * 1.5))
            elif blink_type == "LETTER SPACE":
                winsound.Beep(self.BEEP_FREQ_SHORT, self.BEEP_DURATION)
                time.sleep(0.05)
                winsound.Beep(self.BEEP_FREQ_SHORT, self.BEEP_DURATION)
            elif blink_type == "WORD SPACE":
                winsound.Beep(self.BEEP_FREQ_SHORT, self.BEEP_DURATION)
                time.sleep(0.05)
                winsound.Beep(self.BEEP_FREQ_SHORT, self.BEEP_DURATION)
                time.sleep(0.05)
                winsound.Beep(self.BEEP_FREQ_SHORT, self.BEEP_DURATION)
        except Exception as exc:
            print(f"Sound error: {exc}")

    def decode_morse(self) -> str:
        morse_string = ''.join(self.morse_buffer)
        words = morse_string.split('|')
        decoded = []
        for word in words:
            letters = word.strip().split(' ')
            decoded_word = [self.MORSE_CODE_DICT.get(code, '?') for code in letters if code]
            if decoded_word:
                decoded.append(''.join(decoded_word))
        return ' '.join(decoded)

    def extract_eyes(self, gray: np.ndarray, face):
        """Return two aligned eye crops ready for CNN."""
        try:
            landmarks = self.predictor(gray, face)
            eyes = []
            for eye_points in (self.LEFT_EYE, self.RIGHT_EYE):
                pts = [landmarks.part(i) for i in eye_points]
                x = max(0, min(p.x for p in pts) - 5)
                y = max(0, min(p.y for p in pts) - 5)
                w = min(gray.shape[1] - x, (max(p.x for p in pts) - min(p.x for p in pts)) + 10)
                h = min(gray.shape[0] - y, (max(p.y for p in pts) - min(p.y for p in pts)) + 10)
                eye = gray[y:y + h, x:x + w]
                if eye.size == 0:
                    return None
                eye = cv2.resize(eye, (64, 64))
                eye = eye.astype(np.float32) / 255.0
                eye_rgb = cv2.cvtColor(eye, cv2.COLOR_GRAY2BGR)
                eyes.append(np.expand_dims(eye_rgb, axis=0))
            return eyes
        except Exception as exc:
            print(f"Eye extraction error: {exc}")
            return None

    def predict_blink(self, left_eye, right_eye) -> bool:
        """Return True if both eyes predicted as closed."""
        try:
            if left_eye is None or right_eye is None:
                return False
            pred_left = self.model.predict(left_eye, verbose=0)[0][0]
            pred_right = self.model.predict(right_eye, verbose=0)[0][0]
            return (pred_left < 0.3 and pred_right < 0.3)
        except Exception as exc:
            print(f"Prediction error: {exc}")
            return False

    def process_blink(self, duration: float):
        """Convert blink duration into a Morse symbol and update state."""
        if duration <= self.DOT_MAX:
            symbol, desc = '.', "DOT"
        elif duration <= self.DASH_MIN:
            symbol, desc = '-', "DASH"
        elif duration <= self.LETTER_SEP:
            symbol, desc = ' ', "LETTER SPACE"
        else:
            symbol, desc = '|', "WORD SPACE"

        self.morse_buffer.append(symbol)
        self.play_sound(desc)
        self.last_blink_duration = duration
        self.last_blink_type = desc
        self.feedback_text = f"Added {desc}"
        self.feedback_timer = time.time()
        self.text_buffer = self.decode_morse()

    # ------------------------- New: Backspace -------------------------- #
    def delete_last(self):
        """Remove the most recent Morse symbol, if any, and refresh text."""
        if not self.morse_buffer:
            self.feedback_text = "Nothing to delete"
        else:
            removed = self.morse_buffer.pop()
            kind = {
                '.': 'DOT',
                '-': 'DASH',
                ' ': 'LETTER SPACE',
                '|': 'WORD SPACE',
            }.get(removed, removed)
            self.feedback_text = f"Removed {kind}"
        self.feedback_timer = time.time()
        self.text_buffer = self.decode_morse()

    # --------------------------- Utilities ----------------------------- #
    def reset(self):
        self.morse_buffer.clear()
        self.text_buffer = ""
        self.feedback_text = "System reset"
        self.feedback_timer = time.time()

    def save_message(self):
        if self.text_buffer.strip():
            timestamp = time.strftime("%Y-%m-%d %H:%M:%S")
            self.message_history.append(f"{timestamp}: {self.text_buffer}")
            self.feedback_text = "Message saved"
            self.feedback_timer = time.time()

    # --------------------------- UI helpers ---------------------------- #
    def display_info(self, frame):
        h, w = frame.shape[:2]
        cv2.putText(frame, "Eye Blink Morse Decoder", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 200, 255), 2)
        cv2.putText(frame, "Short: .  Medium: -  Long: Space  XLong: Word", (10, 70),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)
        cv2.putText(frame, "Keys: q‑quit  r‑reset  s‑save  b‑backspace", (10, 95),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.55, (180, 180, 180), 1)

        face_status = "Face: DETECTED" if self.face_detected else "Face: NOT DETECTED"
        face_color = (0, 255, 0) if self.face_detected else (0, 0, 255)
        cv2.putText(frame, face_status, (w - 200, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, face_color, 2)

        cv2.putText(frame, f"Morse: {''.join(self.morse_buffer[-50:])}", (10, 125),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)
        cv2.putText(frame, f"Text: {self.text_buffer[-30:]}", (10, 150),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 1)

        if self.last_blink_duration > 0:
            cv2.putText(frame, f"Last: {self.last_blink_duration:.2f}s ({self.last_blink_type})", (10, 175),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)

        # Timed feedback (3 s)
        if self.feedback_text and (time.time() - self.feedback_timer < 3):
            cv2.putText(frame, self.feedback_text, (10, 205), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)

        y0 = 235
        for i, msg in enumerate(list(self.message_history)[-5:]):
            cv2.putText(frame, msg, (10, y0 + i * 25), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (200, 200, 200), 1)

    # ------------------------------ Main ------------------------------- #
    def run(self):
        cap = cv2.VideoCapture(0)
        if not cap.isOpened():
            print("Error: Could not open video stream.")
            return

        print("Press 'q' to quit, 'r' to reset, 's' to save message, 'b' to backspace")

        try:
            while True:
                ret, frame = cap.read()
                if not ret:
                    break

                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                faces = self.detector(gray)
                self.face_detected = len(faces) > 0

                if self.face_detected:
                    eyes = self.extract_eyes(gray, faces[0])
                    blink_detected = self.predict_blink(*eyes) if eyes else False
                else:
                    blink_detected = False

                now = time.time()
                if self.current_state == "IDLE" and blink_detected:
                    self.blink_start_time = now
                    self.current_state = "BLINKING"
                elif self.current_state == "BLINKING" and not blink_detected:
                    self.process_blink(now - self.blink_start_time)
                    self.blink_start_time = None
                    self.current_state = "IDLE"

                self.display_info(frame)
                cv2.imshow("Eye Blink Morse Decoder", frame)

                key = cv2.waitKey(1) & 0xFF
                if key == ord('q'):
                    break
                elif key == ord('r'):
                    self.reset()
                elif key == ord('s'):
                    self.save_message()
                elif key == ord('b') or key == 8:  # 'b' or Backspace
                    self.delete_last()

        finally:
            cap.release()
            cv2.destroyAllWindows()


if __name__ == "__main__":
    EyeBlinkMorseDecoder().run()


Press 'q' to quit, 'r' to reset, 's' to save message, 'b' to backspace


In [269]:
# Importation des bibliothèques nécessaires
import cv2                      # Pour le traitement d'image (OpenCV)
import time                     # Pour la gestion du temps
from collections import deque   # Pour stocker l'historique des messages

import numpy as np              # Pour le traitement des matrices/images
from tensorflow.keras.models import load_model  # Pour charger le modèle CNN
from ultralytics import YOLO    # Pour la détection des yeux avec YOLOv8
import winsound                 # Pour émettre des bips audio (Windows uniquement)


class EyeBlinkMorseDecoderYOLO:
    def __init__(self, yolo_weights="best.pt", cnn_weights="eye_blink_cnn_model.h5"):
        self.yolo = YOLO(yolo_weights)    # Chargement du modèle YOLO
        self.cnn = load_model(cnn_weights, compile=False)  # Chargement du modèle CNN
        # Dictionnaire du code Morse pour décodage
        self.MORSE_CODE_DICT = {
            '.-': 'A', '-...': 'B', '-.-.': 'C', '-..': 'D', '.': 'E', '..-.': 'F',
            '--.': 'G', '....': 'H', '..': 'I', '.---': 'J', '-.-': 'K', '.-..': 'L',
            '--': 'M', '-.': 'N', '---': 'O', '.--.': 'P', '--.-': 'Q', '.-.': 'R',
            '...': 'S', '-': 'T', '..-': 'U', '...-': 'V', '.--': 'W', '-..-': 'X',
            '-.--': 'Y', '--..': 'Z', '-----': '0', '.----': '1', '..---': '2',
            '...--': '3', '....-': '4', '.....': '5', '-....': '6', '--...': '7',
            '---..': '8', '----.': '9'
        }
        # Seuils pour classer les clignements
        self.DOT_MAX = 0.6   # Clignement court = point
        self.DASH_MIN = 1.0  # Clignement moyen = trait
        self.LETTER_SEP = 1.8  # Pause entre lettres
        self.WORD_SEP = 2.5    # Pause entre mots

        # Paramètres des sons
        self.BEEP_FREQ_SHORT = 800
        self.BEEP_FREQ_LONG = 600
        self.BEEP_DURATION = 100  # Durée du bip
        # Variables d'état
        self.blink_start = None
        self.state = "IDLE"

        self.morse_buffer = []    # Stocke les symboles Morse (. - |)
        self.text_buffer = ""     # Texte décodé
        self.message_history = deque(maxlen=10)   # Historique des messages

        # Feedback pour l'utilisateur
        self.last_blink_type = ""
        self.last_blink_duration = 0.0
        self.feedback_text = ""
        self.feedback_timer = 0.0
        self.eyes_detected = False

    def _classify_eye(self, crop):
        if crop.size == 0:
            return False
        crop = cv2.cvtColor(crop, cv2.COLOR_BGR2GRAY)    # Convertir en niveaux de gris
        crop = cv2.resize(crop, (64, 64)).astype(np.float32) / 255.0   # Redimensionner + normaliser
        crop = np.expand_dims(cv2.cvtColor(crop, cv2.COLOR_GRAY2BGR), axis=0)    # Ajouter dimension batch
        pred = self.cnn.predict(crop, verbose=0)[0][0]  # Prédiction
        return pred < 0.3    # Si la prédiction est < 0.3, œil considéré comme fermé

    def _play_sound(self, blink_type):
        try:
            if blink_type == "DOT":
                winsound.Beep(self.BEEP_FREQ_SHORT, self.BEEP_DURATION)
            elif blink_type == "DASH":
                winsound.Beep(self.BEEP_FREQ_LONG, int(self.BEEP_DURATION * 1.5))
            elif blink_type == "LETTER SPACE":
                winsound.Beep(self.BEEP_FREQ_SHORT, self.BEEP_DURATION)
                time.sleep(0.05)
                winsound.Beep(self.BEEP_FREQ_SHORT, self.BEEP_DURATION)
            elif blink_type == "WORD SPACE":
                for _ in range(3):
                    winsound.Beep(self.BEEP_FREQ_SHORT, self.BEEP_DURATION)
                    time.sleep(0.05)
        except Exception as exc:
            print(f"Sound error: {exc}")

    def _decode_morse(self):
        morse_string = ''.join(self.morse_buffer)
        words = morse_string.split('|')
        decoded_words = []
        for word in words:
            letters = word.strip().split(' ')
            decoded_letters = [self.MORSE_CODE_DICT.get(code, '?') for code in letters if code]
            if decoded_letters:
                decoded_words.append(''.join(decoded_letters))
        return ' '.join(decoded_words)

    def _process_blink(self, duration: float):
        if duration <= self.DOT_MAX:
            symbol, blink_type = '.', "DOT"
        elif duration <= self.DASH_MIN:
            symbol, blink_type = '-', "DASH"
        elif duration <= self.LETTER_SEP:
            symbol, blink_type = ' ', "LETTER SPACE"
        elif duration <= self.WORD_SEP:
            symbol, blink_type = ' ', "LETTER SPACE"
        else:
            symbol, blink_type = '|', "WORD SPACE"

        self.morse_buffer.append(symbol)
        self.last_blink_type = blink_type
        self.last_blink_duration = duration
        self.feedback_text = f"Added {blink_type}"
        self.feedback_timer = time.time()
        self._play_sound(blink_type)
        self.text_buffer = self._decode_morse()

    def _delete_last_symbol(self):
        if self.morse_buffer:
            removed = self.morse_buffer.pop()
            self.feedback_text = f"Deleted: '{removed}'"
            self.feedback_timer = time.time()
            self.text_buffer = self._decode_morse()

    def _delete_last_word(self):
        try:
            last_bar = len(self.morse_buffer) - 1 - self.morse_buffer[::-1].index('|')
            removed = self.morse_buffer[last_bar + 1:]
            del self.morse_buffer[last_bar + 1:]
        except ValueError:
            removed = self.morse_buffer[:]
            self.morse_buffer.clear()

        self.feedback_text = f"Deleted word: {''.join(removed) or '(none)'}"
        self.feedback_timer = time.time()
        self.text_buffer = self._decode_morse()

    def _display_overlay(self, frame):
        # Affichage de l'interface utilisateur (états, texte, morse, instructions)
        h, w = frame.shape[:2]

        cv2.putText(frame, "Eye Blink Morse Decoder (YOLOv8)", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 200, 255), 2)

        cv2.putText(frame,
                    "Short: .  Medium: -  Long: Space  XLong: Word",
                    (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 0.6,
                    (255, 255, 255), 1)

        cv2.putText(frame,
                    "Keys: q-quit  r-reset  s-save  x-backspace  d-del word",
                    (10, 95), cv2.FONT_HERSHEY_SIMPLEX, 0.55,
                    (180, 180, 180), 1)

        eye_status = "Eyes: DETECTED" if self.eyes_detected else "Eyes: NOT DETECTED"
        eye_color = (0, 255, 0) if self.eyes_detected else (0, 0, 255)
        cv2.putText(frame, eye_status,
                    (w - 200, 30), cv2.FONT_HERSHEY_SIMPLEX,
                    0.6, eye_color, 2)

        cv2.putText(frame,
                    f"Morse: {''.join(self.morse_buffer[-50:])}",
                    (10, 125), cv2.FONT_HERSHEY_SIMPLEX, 0.6,
                    (255, 255, 255), 1)

        cv2.putText(frame,
                    f"Text: {self.text_buffer[-30:]}",
                    (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 0.6,
                    (0, 255, 0), 1)

        if self.last_blink_duration:
            cv2.putText(frame,
                        f"Last: {self.last_blink_duration:.2f}s ({self.last_blink_type})",
                        (10, 175), cv2.FONT_HERSHEY_SIMPLEX, 0.6,
                        (0, 255, 255), 2)

        if self.feedback_text and (time.time() - self.feedback_timer < 3):
            cv2.putText(frame, self.feedback_text, (10, 205),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)

        y0 = 235
        for i, msg in enumerate(list(self.message_history)[-5:]):
            cv2.putText(frame, msg, (10, y0 + i * 25),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                        (200, 200, 200), 1)

    def _reset(self):
        # Réinitialise les buffers de morse et texte, et affiche un message de confirmation
        self.morse_buffer.clear()
        self.text_buffer = ""
        self.feedback_text = "System reset"
        self.feedback_timer = time.time()

    def _save_message(self):
        # Sauvegarde le texte courant avec horodatage dans l'historique si non vide
        if self.text_buffer.strip():
            ts = time.strftime("%Y-%m-%d %H:%M:%S")
            self.message_history.append(f"{ts}: {self.text_buffer}")
            self.feedback_text = "Message saved"
            self.feedback_timer = time.time()

    def run(self):
        # Lance la boucle principale de détection et décodage
        cap = cv2.VideoCapture(0)
        if not cap.isOpened():
            print("Error: Could not open webcam.")
            return

        print("Press 'q' quit | 'r' reset | 's' save | 'x' backspace | 'd' delete-word")

        try:
            while True:
                ok, frame = cap.read()
                if not ok:
                    continue

                ts = time.time()
                yolo_results = self.yolo.predict(source=frame, save=False, verbose=False)
                boxes = (yolo_results[0].boxes.xyxy.cpu().numpy()
                         if yolo_results and yolo_results[0].boxes is not None else [])

                closed_flags = []
                self.eyes_detected = len(boxes) > 0

                for box in boxes:
                    # Extraction des coordonnées et découpage de la région oculaire
                    x1, y1, x2, y2 = map(int, box)
                    x1 = max(0, x1); y1 = max(0, y1)
                    x2 = min(frame.shape[1], x2); y2 = min(frame.shape[0], y2)
                    # Classification open/closed avec le CNN
                    # Dessin des rectangles et étiquettes
                    eye_crop = frame[y1:y2, x1:x2]
                    if eye_crop.size == 0:
                        continue

                    is_closed = self._classify_eye(eye_crop)
                    closed_flags.append(is_closed)

                    label = "closed" if is_closed else "open"
                    color = (0, 0, 255) if is_closed else (0, 255, 0)
                    cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                    cv2.putText(frame, label, (x1, y1 - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)

                blink_detected = all(closed_flags) if closed_flags else False

                if self.state == "IDLE" and blink_detected:
                    self.blink_start = ts
                    self.state = "BLINKING"
                elif self.state == "BLINKING" and not blink_detected and self.blink_start:
                    duration = ts - self.blink_start
                    self._process_blink(duration)
                    self.blink_start = None
                    self.state = "IDLE"

                self._display_overlay(frame)
                cv2.imshow("Eye Blink Morse Decoder", frame)

                key = cv2.waitKey(1) & 0xFF
                if key == ord('q'):
                    break
                elif key == ord('r'):
                    self._reset()
                elif key == ord('s'):
                    self._save_message()
                elif key == ord('x'):
                    self._delete_last_symbol()
                elif key == ord('d'):
                    self._delete_last_word()

        finally:
            cap.release()
            cv2.destroyAllWindows()


if __name__ == "__main__":
    app = EyeBlinkMorseDecoderYOLO()
    app.run()


Press 'q' quit | 'r' reset | 's' save | 'x' backspace | 'd' delete-word
